In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
import torch_geometric.transforms as T
from torch_geometric.utils import from_networkx
from torch_geometric.utils import dense_to_sparse
from torch_geometric.nn import GCNConv
import networkx as nx

In [2]:
# Read from csvs without second list in daughter columns (30 s)
training_df = pd.read_csv('train_data.csv')
testing_df = pd.read_csv('test_data.csv')

In [3]:
# Duplicate Jet PT and Eta elements as a list the same length as the number of daughters in each row
def duplicate(df, cols, n_col):
    def duplicate_value(row, col, n_col):
        value = row[col]
        if isinstance(value, float):
            num_daughters = row[n_col]
            return [value] * num_daughters
        return value

    for col in cols:
        df[col] = df.apply(lambda row: duplicate_value(row, col, n_col), axis=1)
    return df

In [4]:
# 17 s
duplicate(training_df, ['Jet0_PT', 'Jet0_Eta'], 'Jet0_nDaughters');
duplicate(testing_df, ['Jet1_PT', 'Jet1_Eta'], 'Jet1_nDaughters');

In [5]:
# Rename DataFrames
df = training_df
df2 = testing_df

In [6]:
# Converts (already duplicated and dropped second array) string into list of floats
def convert_to_lists(df, d_cols):
    for col in d_cols:
        df[col] = df[col].apply(lambda x: [float(num) for num in x.strip('[]').split(',')] if isinstance(x, str) else x)
    return df

In [8]:
# Define features from daughter columns + overall kinematics (already duplicated)
train_d_cols = list(df[df.columns[df.columns.str.contains("_Daughters")]])
test_d_cols = list(df2[df2.columns[df2.columns.str.contains("_Daughters")]])

train_f_cols = train_d_cols + ['Jet0_Eta', 'Jet0_PT']
test_f_cols = test_d_cols + ['Jet1_Eta', 'Jet1_PT']

In [9]:
# Clean up data from str to list of floats (2 min)
convert_to_lists(df, train_d_cols);
convert_to_lists(df2, test_d_cols);

In [10]:
# Creates list of all node features (100115647)
def features_list(df, features_cols):
    node_features = []
    for i, row in df.iterrows():
        for col in features_cols:
            node = df.at[i, col]
            if isinstance(node, list):
                node_features.extend(node)
            else:
                node_features.append(node)
    return node_features

In [11]:
# Gets [num_nodes, num_features] to create tensor
def node_features(df, features_cols):
    node_features_list = []
    node_features = []
    for i, row in df.iterrows():
        row_nodes = []
        for col in features_cols:
            feature = df.at[i, col]
            if isinstance(feature, list):
                node_features.extend(feature)
                if len(row_nodes) < len(feature):
                    row_nodes.extend([[] for _ in range(len(feature) - len(row_nodes))])
                for j, val in enumerate(feature):
                    row_nodes[j].append(val)
            else:
                node_features.append(feature)
                for node in row_nodes:
                        node.append(feature)
        node_features_list.extend(row_nodes)
    return node_features_list

In [12]:
features = node_features(df, train_f_cols)

In [13]:
x = torch.tensor(features, dtype = torch.float)

In [14]:
x.shape # [num_nodes, num_features]

torch.Size([3229537, 31])

In [15]:
# Number of nodes
len(features)

3229537

In [16]:
# Number of nodes = total number of daughters
num_nodes = df['Jet0_nDaughters'].sum()
print(num_nodes)

3229537


In [17]:
# Number of features
num_features = len(train_f_cols)
print(num_features)

31


# 3229537 Nodes, 31 Features

# Trying to create data object without NetworkX

In [31]:
# Function to create the Data object
def create_data_object(df, features_cols):

    x = torch.tensor(features, dtype = torch.float)

    num_nodes = len(features)
    adj = torch.ones((num_nodes, num_nodes))  # Fully connected graph
    edge_index = adj.nonzero(as_tuple=False).t().contiguous()

    y = torch.ones(num_nodes, dtype=torch.long)
    train_mask = torch.ones(num_nodes, dtype=torch.bool)
    test_mask = torch.ones(num_nodes, dtype=torch.bool)

    data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, test_mask=test_mask)

    return data

In [ ]:
# crashes kernel
data_obj = create_data_object(df, train_f_cols)

In [ ]:
def get_data_object(df, features_cols, nd_col):
    adj = torch.ones((num_nodes, num_nodes))  # Fully connected graph
    edge_index = dense_to_sparse(adj)[0]

    y = torch.ones(num_nodes, dtype=torch.long)
    train_mask = torch.ones(num_nodes, dtype=torch.bool)
    test_mask = torch.ones(num_nodes, dtype=torch.bool)

    data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, test_mask=test_mask)

    return data

# Using NetworkX

In [18]:
# Mine - does not quite work

# Create Data object with fully connected nodes
def create_data_object(df, features_cols):

    features = node_features(df, features_cols)

    G = nx.Graph()
    
    # Number of nodes
    num_nodes = len(features)

    # Add nodes with features to NetworkX graph
    for i, feature in enumerate(features):
        G.add_node(i, features=torch.tensor(feature, dtype=torch.float))

    # Fully connected nodes (add edges)
    nodes = list(G.nodes())
    for i, node1 in enumerate(nodes):
        for node2 in nodes[i+1:]:
            G.add_edge(node1, node2)

    # Convert NetworkX graph to PyTorch Geometric Data object
    data = T.from_networkx(G)

    # Set node features, labels, and masks
    data.x = torch.stack([G.nodes[i]['feature'] for i in range(num_nodes)])
    data.y = torch.ones(data.num_nodes, dtype=torch.long)
    data.train_mask = torch.ones(data.num_nodes, dtype=torch.bool)
    data.test_mask = torch.ones(data.num_nodes, dtype=torch.bool)

    return data

In [19]:
train_data = create_data_object(df, train_f_cols)

In [ ]:
print("Number of nodes:", train_data.num_nodes)
print("Node features tensor shape:", train_data.x.shape)
print("First node feature:", train_data.x[0])
print("Labels shape:", train_data.y.shape)
print("Train mask shape:", train_data.train_mask.shape)
print("Test mask shape:", train_data.test_mask.shape)

In [ ]:
test_data = create_data_object(df2, test_f_cols)

In [ ]:
test_data

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

In [ ]:
# Add nodes (each daughter = one node)
    for i, row in df.iterrows():
        for col in features_cols:
            for n in df.at[i,col]:
                G.add_node(n)
    
# Fully connected nodes
    for i, node1 in enumerate(nodes):
        for node2 in nodes[i+1:]:
            G.add_edge(node1, node2)
    
    node_features = torch.tensor(node_features, dtype=torch.float).T

# Training Model

In [ ]:
# Define GNN model
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(train_graph.num_node_features, 64)
        self.conv2 = GCNConv(64, 2)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        return F.log_softmax(x, dim=1)

model = GNN()

In [ ]:
# Training parameters
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train(model, data, optimizer, criterion):
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
    return loss.item()

for epoch in range(1):
    loss = train(model, train_graph, optimizer, criterion)
    print(f'Epoch {epoch+1}, Loss: {loss}')

# Evaluation
def test(model, data):
    model.eval()
    for data in test_loader:
        _, pred = model(data).max(dim=1)
        correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
        acc = correct / data.test_mask.sum().item()
    return acc

accuracy = test(model, test_graph)
print(f'Accuracy: {accuracy}')

# ROC Curve

In [ ]:
output = model(test_graph)
_, pred = output.max(dim=1)
true_labels = test_graph.y.numpy().flatten()
y_score = output[:, 1].detach().numpy()

In [ ]:
fpr, tpr, thresholds = roc_curve(true_labels, y_score)
auc = auc(fpr, tpr)

In [ ]:
back_rej = 1 - fpr
sig_eff = 1 - tpr

print(f'Background Rejection: {back_rej}')
print(f'Signal Efficiency: {sig_eff}')
print(f'AUC: {auc}')

In [ ]:
plt.plot(back_rej, sig_eff, color='purple', label='ROC Curve')
plt.plot([0, 1], [1, 0], color='grey', linestyle='--', label='baseline')
plt.plot([], [], ' ', label=f'AUC: {auc}')
plt.xlim([0.0, 1.01])
plt.ylim([0.0, 1.01])
plt.ylabel('Background Rejection')
plt.xlabel('Signal Efficiency' )
plt.title('Receiver Operating Characteristics (ROC)')
plt.legend(loc='lower left')
plt.show()